# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('../output_data/weather_data.csv')
weather_data.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Litovko,100,RU,1603419349,73,49.24,135.17,39.04,5.53
1,1,Tiksi,84,RU,1603419351,92,71.69,128.87,14.18,11.16
2,2,Hobart,75,AU,1603419268,55,-42.88,147.33,66.99,9.17
3,3,Dikson,79,RU,1603419353,94,73.51,80.55,20.84,8.19
4,4,Cape Town,0,ZA,1603419286,93,-33.93,18.42,63.00,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data[["Lat", "Lng"]]

humidity = weather_data["Humidity"].astype(float)

fig = gmaps.figure()

heatmap = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_types = pd.DataFrame(weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types["Max Temp"] <= 80) & (new_types["Max Temp"] > 70)
wind_speed = new_types["Wind Speed"] < 10
cloudiness = new_types["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
23,San Juan,70.29,8.03,0
85,Faanui,79.81,8.70,0
104,Salalah,75.20,3.36,0
154,Kiryat Gat,72.00,1.12,0
169,Dutse,71.11,3.18,0
190,Carnarvon,78.80,6.93,0
260,Sabla,78.71,4.90,0
265,Jiwani,77.50,8.95,0
312,Morondava,75.83,8.46,0
349,Dubai,75.00,3.36,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# weeks = training_df["Membership (Days)"]/7
# training_df["Membership (Weeks)"] = weeks
# training_df.head()

hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Litovko,RU,49.24,135.17,
1,Tiksi,RU,71.69,128.87,
2,Hobart,AU,-42.88,147.33,
3,Dikson,RU,73.51,80.55,
4,Cape Town,ZA,-33.93,18.42,
...,...,...,...,...,...
570,Imeni Poliny Osipenko,RU,52.42,136.49,
571,Walvis Bay,NaN,-22.96,14.51,
572,Smithers,CA,54.78,-127.17,
573,Rosario,AR,-32.95,-60.64,


In [7]:
target_coord = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coord,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
pprint(response.json(), depth=1)
hotel_df.iloc[0][0]

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


'Litovko'

In [ ]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)


In [ ]:
hotel_name

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
